In [1]:
.include header

Copiarlo a mano

In [ ]:
%%file header/polynomial.h
#pragma once

#include <vector>
#include <iostream>
#include <stdexcept>
#include <memory>
#include <string>
#include <set>

#include "eliastocco_namespace_polynomial.h"
#include "default_basis.h"
#include "basis.h"
#include "evaluate.h"
//#include "Rosetta-Code-fft.h"

namespace eliastocco {

    namespace polynomial {
    
            template < class type , t_dim dimension , t_dim codimension >
                class polynomial {

                    #define thisclass polynomial<type,dimension,codimension>    
                    
                    public :
                        typedef t_var<type,codimension> Type_out;
                        typedef t_var<type,  dimension> Type_in;

                    //default constructor
                    public :
                        thisclass () noexcept 
                            : thisclass( &basis_function_default < type,dimension > ){};  

                    public :
                        thisclass ( basis<type,dimension> * user_basis_function ) noexcept
                            : thisclass ( t_list<Type_out,dimension> () , 
                                            user_basis_function ){};
                                            
                 /*   public :
                        thisclass ( t_coeff_array<type,dimension,codimension> coe,
                                    basis<type,dimension> * user_basis_function ,
                                    t_eval<type,dimension,codimension> user_evaluate ) noexcept
                            : thisclass ( t_list<Type_out,dimension> () , 
                                            user_basis_function , user_evaluate ){                                            
                                            
                                        _coeffs = coe ;
                                        
                                    };    
                                         */   
                                        
                                            
                     public :
                        thisclass ( basis<type,dimension> * user_basis_function ,
                                    t_eval<type,dimension,codimension> user_evaluate ) noexcept
                            : thisclass ( t_list<Type_out,dimension> () , 
                                            user_basis_function , user_evaluate ){};

                    //list : lista dei coefficienti
                    //user constructor                 
                    public :
                        thisclass (const t_list<Type_out,dimension> list) noexcept 
                            : thisclass( list , 
                                    &eliastocco :: polynomial :: basis_function_default < type,dimension > ){};
                                    
 
                         
                    public :
                        thisclass ( const t_list<Type_out,dimension> list,
                                        basis<type,dimension> * user_basis_function ) noexcept 
                                : thisclass ( list , 
                                    user_basis_function ,
                                        eliastocco :: polynomial :: 
                                            default_polynomial_evaluation<type,dimension,codimension> ) {};
                        
                //user constructor
                    public :
                        thisclass ( const t_list<Type_out,dimension> list,
                                        basis<type,dimension> * user_basis_function ,
                                        const t_eval<type,dimension,codimension> user_evaluate ) noexcept 
                            : _basis_function (user_basis_function),
                                _coeffs(),
                                evaluate_lambda(user_evaluate) {  

                            for ( auto it = list.begin(); it != list.end(); ++it){ _coeffs.insert((*it)); }

                        };

//********************************************************************************************************************

                    public :
                        Type_out get ( const t_deg < dimension > index , const bool check = true ) const { 
                        
                            //Type_out a = Type_out();
                            try { 
                                return _coeffs.at(index);                                 
                            } catch ( std::out_of_range e ){  
                            
                                if ( check ){
                                    std :: cout << "error : polynomial.get" << std :: endl;
                                    std :: cout << e.what()<< std :: endl;  
                                }
                                return Type_out();
                            } 
                            //return a;
                            
                        };
                 
                    public : // piccola modifica
                        bool set (const t_deg < dimension > index, const Type_out value , const bool check = true ) {

                            try { 
                                (*_basis_function)[index]; 
                            } catch ( std::out_of_range e ){  
                                if ( check ){
                                    std :: cout << "error : polynomial.set" << std :: endl;
                                    std :: cout << e.what()<< std :: endl;  
                                    return false;
                                }
                            } 

                            _coeffs[index]=value; 

                            return true;

                        };
                        
//********************************************************************************************************************
                     public :
                        inline void clear () { _coeffs.clear () ; }
                        
//********************************************************************************************************************

                    public :
                        inline auto coeffs () const { return _coeffs; }
                        inline auto basis () const { return *_basis_function; }
                       
                        template < template < class ... > class vect = std :: vector >
                            vect < Type_out > all_coeffs () const {
                        
                                vect < Type_out >  output (_basis_function->size());

                                for ( auto i = 0 ; i < output.size() ; ++ i ){                            
                                    output[i] = get(i,false);                                
                                }

                                return output;
                        
                        };
//********************************************************************************************************************

                     public : 
                        friend thisclass multiply 
                            (   const thisclass left ,
                                const thisclass right )  {
                                                                      
                            //ridefinisco il prodotto tra polinomi
                            // I tried the FFT way but I failed

                            // dovrei controllare che i polinomi siano definiti rispetto alla stessa base
                            if ( left._basis_function != right._basis_function ) {
                                assert ("polynomial :: operator * error; coeff array with different basis" );
                            }
                            auto basis = left._basis_function ;
                            // questo metodo di moltiplicare i polinomi funziona solo per polinomi definiti
                            // rispetto ad una base di monomi
                            // per adesso non aggiungo nessun controllo sul "tipo di base"
                            // perché dovrei implementare ancora tutto

                            // la condizione su dimension può essere indebolita
                            // ma per ora implementiamo la moltiplicazione fra polinomi
                            // solo nel caso 1D 
                            static_assert ( codimension == 1 and dimension == 1 );

                            // creo un polinomio e gli assegno i coefficienti calcolati
                            // solo se questi non sono nulli, 
                            // per ottimizzare spazio e rendere l'output più leggibile
                            thisclass output ( basis ) ;

                            // ciclo sui coefficienti dei due polinomi
                            auto L_c = left.coeffs();
                            auto R_c = right.coeffs();

                            // ciclo sui coefficienti di left
                            for ( auto l = L_c.begin() ; l != L_c.end() ; ++l ){

                                auto l_deg   = (*l).first;
                                auto l_coeff = (*l).second;

                                // ciclo sui coefficienti di right
                                for ( auto r = R_c.begin() ; r != R_c.end() ; ++r ){

                                    auto r_deg   = (*r).first;
                                    auto r_coeff = (*r).second;

                                    auto degree = l_deg + r_deg;

                                    auto prev = output.get(degree,false);

                                    output.set(degree, prev + l_coeff*r_coeff );

                                }                        
                            }

                            return output;
                        
                        };                       
                            
                    //polynomial multiplication
                    public :
                        //typename std::enable_if < codimension == 1 and dimension == 1 , thisclass > :: type 
                        friend thisclass operator * ( const thisclass left , const thisclass right ){                        
                            return multiply(left,right) ;
                        }; 
                        
//********************************************************************************************************************
                    
                    // moltiplicazione per uno scalare
                    public :
                        friend thisclass operator * ( const type factor , const thisclass right ){                        
                            
                            auto output = right;                            
                            auto coeffs = output.coeffs();
                            
                            for ( auto it = coeffs.begin() ; it != coeffs.end() ; ++it ){                                
                                auto deg = (*it).first;
                                auto coe = (*it).second;
                                output.set(deg, coe*factor);                            
                            }
                            
                            return output;
                            
                        };
                            
//********************************************************************************************************************
                                            
                    // somma tra due polinomi
                    public :
                        friend thisclass operator + ( const thisclass left , const thisclass right ){ 
                            //std :: cout << "polynomial :: operator +" << std :: endl;
                            // dovrei controllare che i polinomi siano definiti rispetto alla stessa base
                            if ( left._basis_function != right._basis_function ) {
                                assert ("polynomial :: operator * error; coeff array with different basis" );
                            }
                            auto basis = left._basis_function ;
                        
                            std :: set < t_deg < dimension > > degree;
                            
                            auto L_c = left.coeffs();
                            auto R_c = right.coeffs();
                            
                            //left
                            for  (auto it = L_c.begin(); it != L_c.end(); ++it ){
                                degree.insert((*it).first);
                            }
                            //right
                            for  (auto it = R_c.begin(); it != R_c.end(); ++it ){
                                degree.insert((*it).first);
                            }
                            
                            //degree contiene tutti e solo i gradi del nuovo polinomio                            
                            thisclass output (basis);
                            
                            for (auto it = degree.begin(); it != degree.end();++it){
                                auto deg = (*it);
                                //auto left_c  = left.get(deg,false);
                                //auto right_c = right.get(deg,false);
                                output.set(deg,0.0/*left_c+right_c*/);
                            }
                            
                            return output;
                            
                        };

//********************************************************************************************************************
                    //valutazione del polinomio in x
                    public :
                        Type_out operator () ( const Type_in x ) const {
                        
                            //std :: cout << "polynomial operator()" << std :: endl;
                        
                            return evaluate_lambda(_coeffs,*_basis_function,x);
                            //auto a = _basis_function -> evaluate(_coeffs,x);
                        
                            //return _basis_function -> evaluate(_coeffs,x);
                            //return (*_basis_function).evaluate (_coeffs,x); 
                        }; 
                        
                    //vectorized polinomial evaluation
                    public :
                    template < template < class ... > class vect = std :: vector >
                        vect <Type_out> operator () ( const vect < Type_in > x ) const { 
                            vect <Type_out> y (x.size());
                            std :: transform( std :: begin(x), std :: end(x), std :: begin(y),
                                [&]( auto a ) {return (*this)(a);});   
                            return y;

                        }; 
                        
//********************************************************************************************************************

                    private :                
                        eliastocco :: polynomial :: basis <type,dimension> * _basis_function;
                        t_coeff_array <type,dimension,codimension> _coeffs;
                        
                    private :
                        t_eval<type,dimension,codimension> evaluate_lambda;

                    #undef thisclass                

                };
                
       
    }
}

In [2]:
#include "polynomial.h"

In [3]:
using namespace eliastocco :: polynomial;

In [4]:
t_eval<double,1,1> b = default_polynomial_evaluation<double,1,1>

In [5]:
basis_function_default <double,1> = monomial_basis<double>(5)

@0x7f98ef820050

In [6]:
auto base =  monomial_basis<double>(5)

In [7]:
polynomial <double,1,1> A (&base)

In [8]:
polynomial <double,1,1> B (&base)

In [9]:
A.set(0,2); // 2
A.set(1,1); // x
B.set(1,1); // x 

true

In [12]:
A({3.0,2.0})

{ 5.0000000, 4.0000000 }

In [11]:
auto base_2D =  monomial_basis<double,2>({5,5})

In [12]:
polynomial <double,2,1> b_2D (&base_2D)

In [13]:
b_2D.set({0,0},1.0);
b_2D.set({1,0},2.0);
b_2D.set({0,1},5.0);

In [14]:
b_2D({4.0,3.0})

24.000000

In [15]:
polynomial <double,2,3> b_2D_to_3D (&base_2D)

In [16]:
b_2D_to_3D.set({0,0},{1.0,3.0,4.0});
b_2D_to_3D.set({1,0},{1.0,3.0,4.0});
b_2D_to_3D.set({0,1},{1.0,3.0,4.0});

In [17]:
b_2D_to_3D({2.0,4.5})

@0x7ffffbab7070

In [10]:
A.coeffs() // 2+x

{ 0 => 2.0000000, 1 => 1.0000000 }

In [11]:
(3*A).coeffs() // 2+x

{ 0 => 6.0000000, 1 => 3.0000000 }

In [12]:
B.coeffs() // x

{ 1 => 1.0000000 }

In [13]:
auto C = A*B; //2x + x^2
C.coeffs()

{ 1 => 2.0000000, 2 => 1.0000000 }

In [14]:
C.coeffs()

{ 1 => 2.0000000, 2 => 1.0000000 }

In [15]:
A.clear();
B.clear();
A.set(2,3); // 3x^2
B.set(1,4); // 4x

true

In [16]:
C=A*B; //12x^3
C.coeffs()

{ 3 => 12.000000 }

In [17]:
auto D = multiply(A,B)

In [18]:
auto E = A + B

In [19]:
A.coeffs()

{ 2 => 3.0000000 }

In [20]:
B.coeffs()

{ 1 => 4.0000000 }

In [21]:
E.coeffs()

{ 1 => 0.0000000, 2 => 0.0000000 }

In [22]:
A.set(1,3.)

true

In [23]:
C=A*B;
C.coeffs()

{ 2 => 12.000000, 3 => 12.000000 }

In [24]:
B.set(0,0);
B.set(1,0);
B.set(2,0);
B.set(3,0);
B.set(4,0)

true

In [25]:
auto BB = B*B

error : polynomial.set
map::at
error : polynomial.set
map::at
error : polynomial.set
map::at
error : polynomial.set
map::at
error : polynomial.set
map::at
error : polynomial.set
map::at


In [26]:
(A*A).coeffs()

{ 2 => 9.0000000, 3 => 18.000000, 4 => 9.0000000 }

Inizializzazione specificando prima tutte le componenti i-esime di tutti i control points

In [28]:
A.set(1,{2.0,3.0,0.0});
A.set(0,{4.0,5.0,6.0})

input_line_52:2:10: error: excess elements in scalar initializer
 A.set(1,{2.0,3.0,0.0});
         ^   ~~~~~~~~~
header/polynomial.h:79:83: note: passing argument to parameter 'value' here
                        bool set (const t_deg < dimension > index, const Type_out value , const bool check = true ) {
                                                                                  ^
input_line_52:3:9: error: excess elements in scalar initializer
A.set(0,{4.0,5.0,6.0})
        ^   ~~~~~~~~~
header/polynomial.h:79:83: note: passing argument to parameter 'value' here
                        bool set (const t_deg < dimension > index, const Type_out value , const bool check = true ) {
                                                                                  ^


Interpreter Error: 

In [29]:
A.set(7,{4.0,5.0,6.0})

input_line_53:2:10: error: excess elements in scalar initializer
 A.set(7,{4.0,5.0,6.0})
         ^   ~~~~~~~~~
header/polynomial.h:79:83: note: passing argument to parameter 'value' here
                        bool set (const t_deg < dimension > index, const Type_out value , const bool check = true ) {
                                                                                  ^


Interpreter Error: 

In [ ]:
B.get(0)

In [ ]:
B.get(0,true)

In [ ]:
B.get(0,false)

In [ ]:
B.set(1,3)

In [ ]:
A.coeffs().size()

In [ ]:
auto v = A.all_coeffs()

In [ ]:
A.all_coeffs()

In [ ]:
typeid(v).name()

In [ ]:
auto V = A.all_coeffs<std::vector>()

In [ ]:
auto vV = A.all_coeffs<std::valarray>()

In [30]:
A(2.0)

18.000000

In [22]:
std :: vector <double> x = {2,3,4,5.0}

In [23]:
auto out = A(x)

In [24]:
auto ret = A({2,3,4})

In [27]:
std :: valarray<std :: array <double,3>> val = A.operator()<std :: valarray>({2,3,4})

In [30]:
std :: vector<std :: array <double,3>> Val = A({2,3,4})

In [31]:
auto VAl = A({2,3,4})

In [32]:
auto VAL = A.operator()<std :: valarray > ({2,3,4})

In [33]:
auto VALL = A.operator()<std :: deque > ({2,3,4})

Inizializzazione specificando le componenti un control points per volta

In [17]:
C.get(0)

@0x7fffdca41980

Valutazione della Bspline

Ricordarsi di inizializzare la base

In [16]:
basis_function_default<double,1> = monomial_basis<double,1>(5);

In [17]:
Bspline :: Bspline<double,1,3> BS

In [18]:
BS.get(0)

error: Bspline.get
map::at


@0x7fffbbd755b0

In [19]:
BS.set(0,{-1,1,0});

In [20]:
BS.get(0)

@0x7fffbc17f310

In [21]:
BS.set(2,{2,3,4});

In [22]:
BS.get(2)[2]

4.0000000

In [23]:
BS.pol[0](2.0)

7.0000000

In [24]:
BS(0.0)

@0x7fffbc5ea1f0

In [25]:
BS({2.0,3.0})

{ @0x7fffbc6e9040, @0x7fffbc6e9058 }

In [26]:
auto vect_Y = BS ( {2,3,4,5})

In [27]:
vect_Y.size()

4

In [28]:
auto get = BS.get(0)

In [29]:
auto get_v = BS.get_v(0)

In [30]:
get_v.size()

3

In [31]:
get.size()

3

In [32]:
BS.set(2,{1,2,3})

true

In [38]:
BS.get(2).at(3)

Standard Exception: array::at: __n (which is 3) >= _Nm (which is 3)

In [ ]:
%%file header/polynomial.h
#pragma once

#include <vector>
#include <iostream>
#include <stdexcept>
#include <memory>
#include <string>
#include <set>

#include "eliastocco_namespace_polynomial.h"
#include "default_basis.h"
#include "basis.h"
#include "evaluate.h"
//#include "Rosetta-Code-fft.h"

namespace eliastocco {

    namespace polynomial {
    
            template < class type , t_dim dimension , t_dim codimension >
                class polynomial {

                    #define thisclass polynomial<type,dimension,codimension>    
                    
                    public :
                        typedef t_var<type,codimension> Type_out;
                        typedef t_var<type,  dimension> Type_in;

                    //default constructor
                    public :
                        thisclass () noexcept 
                            : thisclass( &basis_function_default < type,dimension > ){};  

                    public :
                        thisclass ( basis<type,dimension> * user_basis_function ) noexcept
                            : thisclass ( t_list<Type_out,dimension> () , 
                                            user_basis_function ){};

                    //list : lista dei coefficienti
                    //user constructor                 
                    public :
                        thisclass (const t_list<Type_out,dimension> list) noexcept 
                            : thisclass( list , 
                                    &eliastocco :: polynomial :: basis_function_default < type,dimension > ){};
                                    
 
                         
                    public :
                        thisclass ( const t_list<Type_out,dimension> list,
                                        basis<type,dimension> * user_basis_function ) noexcept 
                                : thisclass ( list , 
                                    user_basis_function ,
                                        eliastocco :: polynomial :: 
                                            default_polynomial_evaluation<type,dimension,codimension> ) {};
 
                    //user constructor
                    public :
                        thisclass ( const t_list<Type_out,dimension> list,
                                        basis<type,dimension> * user_basis_function ,
                                        const t_eval<type,dimension,codimension> user_evaluate ) noexcept 
                            : _basis_function (user_basis_function),
                                _coeffs(),
                                evaluate_lambda(user_evaluate) {  

                            for ( auto it = list.begin(); it != list.end(); ++it){ _coeffs.insert((*it)); }

                        };

//********************************************************************************************************************

                    public :
                        Type_out get ( const t_deg < dimension > index , const bool check = true ) const { 
                        
                            //Type_out a = Type_out();
                            try { 
                                return _coeffs.at(index);                                 
                            } catch ( std::out_of_range e ){  
                            
                                if ( check ){
                                    std :: cout << "error : polynomial.get" << std :: endl;
                                    std :: cout << e.what()<< std :: endl;  
                                }
                                return Type_out();
                            } 
                            //return a;
                            
                        };
                 
                    public : // piccola modifica
                        bool set (const t_deg < dimension > index, const Type_out value , const bool check = true ) {

                            try { 
                                (*_basis_function)[index]; 
                            } catch ( std::out_of_range e ){  
                                if ( check ){
                                    std :: cout << "error : polynomial.set" << std :: endl;
                                    std :: cout << e.what()<< std :: endl;  
                                    return false;
                                }
                            } 

                            _coeffs[index]=value; 

                            return true;

                        };
                        
//********************************************************************************************************************
                     public :
                        inline void clear () { _coeffs.clear () ; }
                        
//********************************************************************************************************************

                    public :
                        inline auto coeffs () const { return _coeffs; }
                        inline auto basis () const { return *_basis_function; }
                       
                        template < template < class ... > class vect = std :: vector >
                            vect < Type_out > all_coeffs () const {
                        
                                vect < Type_out >  output (_basis_function->size());

                                for ( auto i = 0 ; i < output.size() ; ++ i ){                            
                                    output[i] = get(i,false);                                
                                }

                                return output;
                        
                        };
//********************************************************************************************************************

                     public : 
                        friend thisclass multiply 
                            (   const thisclass left ,
                                const thisclass right )  {
                                                                      
                            //ridefinisco il prodotto tra polinomi
                            // I tried the FFT way but I failed

                            // dovrei controllare che i polinomi siano definiti rispetto alla stessa base
                            if ( left._basis_function != right._basis_function ) {
                                assert ("polynomial :: operator * error; coeff array with different basis" );
                            }
                            auto basis = left._basis_function ;
                            // questo metodo di moltiplicare i polinomi funziona solo per polinomi definiti
                            // rispetto ad una base di monomi
                            // per adesso non aggiungo nessun controllo sul "tipo di base"
                            // perché dovrei implementare ancora tutto

                            // la condizione su dimension può essere indebolita
                            // ma per ora implementiamo la moltiplicazione fra polinomi
                            // solo nel caso 1D 
                            static_assert ( codimension == 1 and dimension == 1 );

                            // creo un polinomio e gli assegno i coefficienti calcolati
                            // solo se questi non sono nulli, 
                            // per ottimizzare spazio e rendere l'output più leggibile
                            thisclass output ( basis ) ;

                            // ciclo sui coefficienti dei due polinomi
                            auto L_c = left.coeffs();
                            auto R_c = right.coeffs();

                            // ciclo sui coefficienti di left
                            for ( auto l = L_c.begin() ; l != L_c.end() ; ++l ){

                                auto l_deg   = (*l).first;
                                auto l_coeff = (*l).second;

                                // ciclo sui coefficienti di right
                                for ( auto r = R_c.begin() ; r != R_c.end() ; ++r ){

                                    auto r_deg   = (*r).first;
                                    auto r_coeff = (*r).second;

                                    auto degree = l_deg + r_deg;

                                    auto prev = output.get(degree,false);

                                    output.set(degree, prev + l_coeff*r_coeff );

                                }                        
                            }

                            return output;
                        
                        };                       
                            
                    //polynomial multiplication
                    public :
                        //typename std::enable_if < codimension == 1 and dimension == 1 , thisclass > :: type 
                        friend thisclass operator * ( const thisclass left , const thisclass right ){                        
                            return multiply(left,right) ;
                        }; 
                        
//********************************************************************************************************************
                    
                    // moltiplicazione per uno scalare
                    public :
                        friend thisclass operator * ( const type factor , const thisclass right ){                        
                            
                            auto output = right;                            
                            auto coeffs = output.coeffs();
                            
                            for ( auto it = coeffs.begin() ; it != coeffs.end() ; ++it ){                                
                                auto deg = (*it).first;
                                auto coe = (*it).second;
                                output.set(deg, coe*factor);                            
                            }
                            
                            return output;
                            
                        };
                            
//********************************************************************************************************************
                                            
                    // somma tra due polinomi
                    public :
                        friend thisclass operator + ( const thisclass left , const thisclass right ){ 
                            //std :: cout << "polynomial :: operator +" << std :: endl;
                            // dovrei controllare che i polinomi siano definiti rispetto alla stessa base
                            if ( left._basis_function != right._basis_function ) {
                                assert ("polynomial :: operator * error; coeff array with different basis" );
                            }
                            auto basis = left._basis_function ;
                        
                            std :: set < t_deg < dimension > > degree;
                            
                            auto L_c = left.coeffs();
                            auto R_c = right.coeffs();
                            
                            //left
                            for  (auto it = L_c.begin(); it != L_c.end(); ++it ){
                                degree.insert((*it).first);
                            }
                            //right
                            for  (auto it = R_c.begin(); it != R_c.end(); ++it ){
                                degree.insert((*it).first);
                            }
                            
                            //degree contiene tutti e solo i gradi del nuovo polinomio                            
                            thisclass output (basis);
                            
                            for (auto it = degree.begin(); it != degree.end();++it){
                                auto deg = (*it);
                                //auto left_c  = left.get(deg,false);
                                //auto right_c = right.get(deg,false);
                                output.set(deg,0.0/*left_c+right_c*/);
                            }
                            
                            return output;
                            
                        };

//********************************************************************************************************************
                    //valutazione del polinomio in x
                    public :
                        Type_out operator () ( const Type_in x ) const {
                        
                            auto a = evaluate_lambda(_coeffs,*_basis_function,x);
                        
                            return a;
                            //return (*_basis_function).evaluate (_coeffs,x); 
                        }; 
                        
                    //vectorized polinomial evaluation
                    public :
                    template < template < class ... > class vect = std :: vector >
                        vect <Type_out> operator () ( const vect < Type_in > x ) const { 
                            vect <Type_out> y (x.size());
                            std :: transform( std :: begin(x), std :: end(x), std :: begin(y),
                                [&]( auto a ) {return (*this)(a);});   
                            return y;

                        }; 
                        
//********************************************************************************************************************

                    private :                
                        eliastocco :: polynomial :: basis <type,dimension> * _basis_function;
                        t_coeff_array <type,dimension,codimension> _coeffs;
                        
                    private :
                        t_eval<type,dimension,codimension> evaluate_lambda;

                    #undef thisclass                

                };
                
       
    }
}